In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit import Model
import plotly.express as px
import scipy.stats as stats
import tqdm
from IPython.display import Image

## 1. Разбегание траекторий
Построим разницу траекторий

In [29]:
dx = pd.read_csv("diffx.csv")
dv = pd.read_csv("diffv.csv")

In [30]:
fig = px.line(x=dx["time"][0:8000],y=dx["diff_x"][0:8000],labels={'x':'Time', 'y':'<dx^2>'})
fig.update_layout(template='plotly_white',title=r'idk')
fig.show()

In [31]:
fig = px.line(x=dv["time"][0:8000],y=dv["diff_v"][0:8000],labels={'x':'Time', 'y':'<dv^2>'})
fig.update_layout(template='plotly_white',title=r'idk')
fig.show()

## Расчет коэффициента самодиффузии

### T=1.0

In [125]:
frames = 999
particles = 1024
dt = 1e-2
x = np.zeros((frames, particles))
y = np.zeros((frames, particles))
z = np.zeros((frames, particles))
vx = np.zeros((frames, particles))
vy = np.zeros((frames, particles))
vz = np.zeros((frames, particles))

### Формула Эйнштейна-Смолуховского
$$ D = \lim_{t \to \infty} \frac{1}{6t}\langle |\pmb{r}_{i}(t) - \pmb{r}_{i}(0)|^{2} \rangle$$

In [126]:
with open("t_10.xyz", "r") as file:
    for frame in tqdm.notebook.trange(frames, desc="Frame"):
        try:
            n = int(file.readline())
        except:
            break
        file.readline()

        coord = []

        for i in range(0,n):
            line = file.readline()
            coord = [float(j) for j in line.split(" ")]

            x[frame][i] = coord[0]
            y[frame][i] = coord[1]
            z[frame][i] = coord[2]

            vx[frame][i] = coord[3]
            vy[frame][i] = coord[4]
            vz[frame][i] = coord[5]

In [127]:
msd = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames - frame):
        msd[frame]+=np.square((x[frame+start]- x[start])).mean()/(frames - frame)
        msd[frame]+=np.square((y[frame+start] - y[start])).mean()/(frames - frame)
        msd[frame]+=np.square((z[frame+start] - z[start])).mean()/(frames - frame)

In [128]:
fig = px.line(x=np.arange(0, frames, 1),y=msd,labels={'x':'Frame', 'y':'<dr^2>'})
fig.update_layout(template='plotly_white',title=r'$T=1.0 \quad \rho=0.7$')
fig.show()

In [130]:
d_es_10 = np.zeros(len(msd))
for frame in range(1, frames):
    d_es_10[frame] = msd[frame]/(6*frame*dt)

In [131]:
fig = px.line(x=np.arange(0, frames, 1),y=d_es_10,labels={'x':'Frame', 'y':'D'})
fig.update_layout(template='plotly_white',title=r'$T=1.0 \quad \rho=0.7$')
fig.show()

### Формула Грина-Кубо
$$ D = \frac{1}{3} \int_0^\infty \langle v_{i}(t) \cdot v_{i}(0) dt\rangle$$

In [132]:
vac = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames-frame):
        vac[frame]+=np.multiply(vx[frame+start],vx[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vy[frame+start],vy[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vz[frame+start],vz[start]).mean()/(3*(frames-frame))

In [133]:
fig = px.line(x=np.arange(0, frames, 1),y=vac,labels={'x':'Frame', 'y':'Velocity autocorelation'})
fig.update_layout(template='plotly_white',title=r'$T=1.0 \quad \rho=0.7$')
fig.show()

In [134]:
d_gk_10 = 0
for i in range(1, frames):
    d_gk_10+=0.5*dt*(vac[i-1]+vac[i])
print(d_gk_10)

0.10662560990633724


### T=1.5

In [138]:
frames = 999
particles = 1024
dt = 1e-2
x = np.zeros((frames, particles))
y = np.zeros((frames, particles))
z = np.zeros((frames, particles))
vx = np.zeros((frames, particles))
vy = np.zeros((frames, particles))
vz = np.zeros((frames, particles))

### Формула Эйнштейна-Смолуховского

In [139]:
with open("t_15.xyz", "r") as file:
    for frame in tqdm.notebook.trange(frames, desc="Frame"):
        try:
            n = int(file.readline())
        except:
            break
        file.readline()

        coord = []

        for i in range(0,n):
            line = file.readline()
            coord = [float(j) for j in line.split(" ")]

            x[frame][i] = coord[0]
            y[frame][i] = coord[1]
            z[frame][i] = coord[2]

            vx[frame][i] = coord[3]
            vy[frame][i] = coord[4]
            vz[frame][i] = coord[5]

In [140]:
msd = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames - frame):
        msd[frame]+=np.square((x[frame+start]- x[start])).mean()/(frames - frame)
        msd[frame]+=np.square((y[frame+start] - y[start])).mean()/(frames - frame)
        msd[frame]+=np.square((z[frame+start] - z[start])).mean()/(frames - frame)

In [141]:
fig = px.line(x=np.arange(0, frames, 1),y=msd,labels={'x':'Frame', 'y':'<dr^2>'})
fig.update_layout(template='plotly_white',title=r'$T=1.5 \quad \rho=0.7$')
fig.show()

In [142]:
d_es_15 = np.zeros(len(msd))
for frame in range(1, frames):
    d_es_15[frame] = msd[frame]/(6*frame*dt)

In [143]:
fig = px.line(x=np.arange(0, frames, 1),y=d_es_15,labels={'x':'Frame', 'y':'D'})
fig.update_layout(template='plotly_white',title=r'$T=1.5 \quad \rho=0.7$')
fig.show()

### Формула Грина-Кубо

In [144]:
vac = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames-frame):
        vac[frame]+=np.multiply(vx[frame+start],vx[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vy[frame+start],vy[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vz[frame+start],vz[start]).mean()/(3*(frames-frame))

In [145]:
fig = px.line(x=np.arange(0, frames, 1),y=vac,labels={'x':'Frame', 'y':'Velocity autocorelation'})
fig.update_layout(template='plotly_white',title=r'$T=1.5 \quad \rho=0.7$')
fig.show()

In [146]:
d_gk_15 = 0
for i in range(1, frames):
    d_gk_15+=0.5*dt*(vac[i-1]+vac[i])
print(d_gk_15)

0.16142514438787164


### T=2.0

In [174]:
frames = 999
particles = 1024
dt = 1e-2
x = np.zeros((frames, particles))
y = np.zeros((frames, particles))
z = np.zeros((frames, particles))
vx = np.zeros((frames, particles))
vy = np.zeros((frames, particles))
vz = np.zeros((frames, particles))

### Формула Эйнштейна-Смолуховского

In [175]:
with open("t_20.xyz", "r") as file:
    for frame in tqdm.notebook.trange(frames, desc="Frame"):
        try:
            n = int(file.readline())
        except:
            break
        file.readline()

        coord = []

        for i in range(0,n):
            line = file.readline()
            coord = [float(j) for j in line.split(" ")]

            x[frame][i] = coord[0]
            y[frame][i] = coord[1]
            z[frame][i] = coord[2]

            vx[frame][i] = coord[3]
            vy[frame][i] = coord[4]
            vz[frame][i] = coord[5]

In [176]:
msd = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames - frame):
        msd[frame]+=np.square((x[frame+start]- x[start])).mean()/(frames - frame)
        msd[frame]+=np.square((y[frame+start] - y[start])).mean()/(frames - frame)
        msd[frame]+=np.square((z[frame+start] - z[start])).mean()/(frames - frame)

In [177]:
fig = px.line(x=np.arange(0, frames, 1),y=msd,labels={'x':'Frame', 'y':'<dr^2>'})
fig.update_layout(template='plotly_white',title=r'$T=2.0 \quad \rho=0.7$')
fig.show()

In [178]:
d_es_20 = np.zeros(len(msd))
for frame in range(1, frames):
    d_es_20[frame] = msd[frame]/(6*frame*dt)

In [179]:
fig = px.line(x=np.arange(0, frames, 1),y=d_es_20,labels={'x':'Frame', 'y':'D'})
fig.update_layout(template='plotly_white',title=r'$T=2.0 \quad \rho=0.7$')
fig.show()

### Формула Грина-Кубо

In [180]:
vac = np.zeros(frames)

for frame in tqdm.notebook.trange(frames, desc="Frame"):
    for start in range(0, frames-frame):
        vac[frame]+=np.multiply(vx[frame+start],vx[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vy[frame+start],vy[start]).mean()/(3*(frames-frame))
        vac[frame]+=np.multiply(vz[frame+start],vz[start]).mean()/(3*(frames-frame))

In [181]:
fig = px.line(x=np.arange(0, frames, 1),y=vac,labels={'x':'Frame', 'y':'Velocity autocorelation'})
fig.update_layout(template='plotly_white',title=r'$T=2.0 \quad \rho=0.7$')
fig.show()

In [183]:
d_gk_20 = 0
for i in range(1, frames):
    d_gk_20+=0.5*dt*(vac[i-1]+vac[i])
print(d_gk_20)

0.23680356009487286
